[View in Colaboratory](https://colab.research.google.com/github/meghamattikalli/lstm/blob/master/lstm_crossval.ipynb)

In [4]:
import os

from google.colab import files
uploaded = files.upload()

Saving five_thousand.csv to five_thousand (1).csv
Saving Indian-Female-Names1.csv to Indian-Female-Names1 (1).csv
Saving Indian-Male-Names1.csv to Indian-Male-Names1 (1).csv


In [6]:

from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np


import io

female = pd.read_csv(io.StringIO(uploaded['Indian-Female-Names1.csv'].decode('utf-8')))

male = pd.read_csv(io.StringIO(uploaded['Indian-Male-Names1.csv'].decode('utf-8')))

eng_word = pd.read_csv(io.StringIO(uploaded['five_thousand.csv'].decode('utf-8')))


#parameters
maxlen = 10
labels = 2

# Importing the dataset
#female= pd.read_csv('Indian-Female-Names1.csv')
#male= pd.read_csv('Indian-Male-Names1.csv')
names=pd.concat([female,male], axis=0)
names= names.apply(lambda x: x.astype(str).str.lower())
names=names.iloc[:,0:1]
names["type"]=1
names=names[names['word'].map(len) > 2]

unwanted=['mr','.','moh.','@','miss','&','mrs','/','ku.','km',',','km0','-','(',')','smt','smt.','`','[','ms','1','2','3','4','5','6','7','8','9','0']
for i in unwanted:
    names['word'] = names.word.str.replace(i, '')
names['word'] = names.word.str.strip()
names=names.drop_duplicates(subset=['word'], keep='first')
names['word'] = names.word.str.partition(" ")


#english words
#eng_word = pd.read_csv('five_thousand.csv')
eng_word = eng_word.apply(lambda x: x.astype(str).str.lower())
eng_word['word'] = eng_word.word.str.replace('-', '')
eng_word['word'] = eng_word.word.str.replace("'", '')
eng_word['word'] = eng_word.word.str.replace("/", ' ')
eng_word['word'] = eng_word.word.str.partition(" ")


#eng_pre = pd.read_csv('prepositions.csv')
eng_word['type'] = 0
#eng_pre['type'] = 0

#concatenate names and english words
dataset=pd.concat([names,eng_word], axis=0)
dataset=dataset.dropna()

dataset.index=[i for i in range(0,len(dataset.index))] #renaming the indices
features = dataset['word']
label = dataset['type']

vocab = set(' '.join([str(i) for i in features]))

vocab.add('END')
len_vocab = len(vocab)

print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(dataset))

char_index = dict((c, i) for i, c in enumerate(vocab))

print(char_index)


#train test split
msk = np.random.rand(len(dataset)) < 0.8
train = dataset[msk]
test = dataset[~msk]


def set_flag(i):
    tmp = np.zeros(28);
    tmp[i] = 1
    return(tmp)
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.word]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.type:
    if i == 0:
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])

np.asarray(train_X).shape

np.asarray(train_Y).shape


{'p', 's', 'o', 'i', 'n', 'j', 'h', 'x', 'z', ' ', 'f', 'd', 'k', 'l', 'e', 't', 'u', 'a', 'r', 'c', 'y', 'g', 'v', 'q', 'm', 'END', 'b', 'w'}
vocab length is  28
length of input is  18716
{'p': 0, 's': 1, 'o': 2, 'i': 3, 'n': 4, 'j': 5, 'h': 6, 'x': 7, 'z': 8, ' ': 9, 'f': 10, 'd': 11, 'k': 12, 'l': 13, 'e': 14, 't': 15, 'u': 16, 'a': 17, 'r': 18, 'c': 19, 'y': 20, 'g': 21, 'v': 22, 'q': 23, 'm': 24, 'END': 25, 'b': 26, 'w': 27}


(14863, 2)

In [0]:

def build_classifier():
  model = Sequential()
  model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
  model.add(Dropout(0.2))
  model.add(LSTM(512, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(2))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  return model



In [10]:
train_X=np.array(train_X)
train_Y=np.array(train_Y)
#test_Y=np.array(test_Y)
#test_X=np.array(test_X)
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 20)
accuracies = cross_val_score(estimator = classifier, X = train_X, y = train_Y, cv = 2, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/20
Epoch 1/20


JoblibInternalError: ignored